# Домашнее задание 4. Поиск аномалий
**Дедлайн 23:59 07.12.2018**



# Задача
[Данные](http://odds.cs.stonybrook.edu/seismic-dataset/) представляют собой показания о сейсмических активностях вулканов. Задача имеет очень важное практическое значение. Ещё недавно эта задача требовала большой квалификации экспертов и была очень сложной. Однако сейчас, в эпоху машинного обучения, за вас учится компьютер. Однако вот незадача: за всю историю геонаблюдений наблюдалось всего несколько (170) землятрясений, в то время как нормальных геосостояний окрестностей наблюдалось очень много (~2500). Перед Вами стоит задача наиболее эффективного поиска аномалий

# Условие
В рамках данной задачи Вам предлагается попробовать следующее:
1. Предложите метрику качества Вашей модели. Можно ли здесь использовать accuracy? Объясните свой выбор.
2. Проверьте гипотезу компактности на этих данных. Какие методы машинного обучения тут могут хорошо работать?
3. Обучите на данных любой классификатор. Хорошо ли он работает? Почему его качество может быть плохим в этой задаче?
4. Обучите методы поиска аномалий из [занятия 6](../Seminars/Lesson_6.ipynb). Добейтесь наилучшего качества. Лучше ли оно, чем supervised-learning? Почему?
5. Визуализируйте полученные наилучшей моделью предсказания. Хороша ли итоговая модель на этих данных?

# Примечание
1. Более подробное описание данных можно найти [тут](https://archive.ics.uci.edu/ml/datasets/seismic-bumps#)
2. Если эти данные Вам не нравятся - можете взять любые другие данные с аномалиями (кроме тироида) [отсюда](http://odds.cs.stonybrook.edu)

# Ссылки
https://dyakonov.org/2017/04/19/поиск-аномалий-anomaly-detection/

# Решение

## 0. Загрузка и предобработка данных

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00266/seismic-bumps.arff -P ../data/.

--2018-11-29 14:00:57--  https://archive.ics.uci.edu/ml/machine-learning-databases/00266/seismic-bumps.arff
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141669 (138K) [text/plain]
data/.: File exists
Saving to: ‘data/./seismic-bumps.arff’

seismic-bumps.arff  100%[===================>] 138,35K   113KB/s    in 1,2s    

2018-11-29 14:00:59 (113 KB/s) - ‘data/./seismic-bumps.arff’ saved [141669/141669]



In [4]:
from scipy.io import arff
data, meta = arff.loadarff('data/seismic-bumps.arff')

In [5]:
data

array([(b'a', b'a', b'N', 15180.,  48., -72., -72., b'a', 0., 0., 0., 0., 0., 0., 0., 0.,    0.,    0., b'0'),
       (b'a', b'a', b'N', 14720.,  33., -70., -79., b'a', 1., 0., 1., 0., 0., 0., 0., 0., 2000., 2000., b'0'),
       (b'a', b'a', b'N',  8050.,  30., -81., -78., b'a', 0., 0., 0., 0., 0., 0., 0., 0.,    0.,    0., b'0'),
       ...,
       (b'b', b'a', b'W', 26960., 540., 101., 112., b'a', 0., 0., 0., 0., 0., 0., 0., 0.,    0.,    0., b'0'),
       (b'a', b'a', b'W', 16130., 322.,   2.,   2., b'a', 0., 0., 0., 0., 0., 0., 0., 0.,    0.,    0., b'0'),
       (b'a', b'a', b'W', 12750., 235., -10., -10., b'a', 0., 0., 0., 0., 0., 0., 0., 0.,    0.,    0., b'0')],
      dtype=[('seismic', 'S1'), ('seismoacoustic', 'S1'), ('shift', 'S1'), ('genergy', '<f8'), ('gpuls', '<f8'), ('gdenergy', '<f8'), ('gdpuls', '<f8'), ('ghazard', 'S1'), ('nbumps', '<f8'), ('nbumps2', '<f8'), ('nbumps3', '<f8'), ('nbumps4', '<f8'), ('nbumps5', '<f8'), ('nbumps6', '<f8'), ('nbumps7', '<f8'), ('nbumps89

In [6]:
meta

Dataset: seismic-bumps
	seismic's type is nominal, range is ('a', 'b', 'c', 'd')
	seismoacoustic's type is nominal, range is ('a', 'b', 'c', 'd')
	shift's type is nominal, range is ('W', 'N')
	genergy's type is numeric
	gpuls's type is numeric
	gdenergy's type is numeric
	gdpuls's type is numeric
	ghazard's type is nominal, range is ('a', 'b', 'c', 'd')
	nbumps's type is numeric
	nbumps2's type is numeric
	nbumps3's type is numeric
	nbumps4's type is numeric
	nbumps5's type is numeric
	nbumps6's type is numeric
	nbumps7's type is numeric
	nbumps89's type is numeric
	energy's type is numeric
	maxenergy's type is numeric
	class's type is nominal, range is ('1', '0')

In [23]:
import pandas as pd
df = pd.DataFrame(data, columns=data.dtype.fields.keys())
df.head()

,seismic,seismoacoustic,shift,genergy,gpuls,gdenergy,gdpuls,ghazard,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,nbumps6,nbumps7,nbumps89,energy,maxenergy,class
0,b'a',b'a',b'N',15180.0,48.0,-72.0,-72.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
1,b'a',b'a',b'N',14720.0,33.0,-70.0,-79.0,b'a',1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2000.0,2000.0,b'0'
2,b'a',b'a',b'N',8050.0,30.0,-81.0,-78.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
3,b'a',b'a',b'N',28820.0,171.0,-23.0,40.0,b'a',1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3000.0,3000.0,b'0'
4,b'a',b'a',b'N',12640.0,57.0,-63.0,-52.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'


Данные сырые и явно требуют предобработки

In [31]:
df['class'] = df['class'].astype(int) # приведём целевую переменную к числовым значениям (1 - аномалия)

Руководствуясь метаинформацией о признаках (переменная meta выше) и таблицей выше, определите и преобразуйте категориальные признаки

In [39]:
categorical_features = [???]
from sklearn.preprocessing import LabelEncoder

transformers = {}
for feature_name in categorical_features:
    transformers[feature_name] = LabelEncoder()
    df[feature_name] = transformers[feature_name].fit_transform(df[feature_name])

df_oh = pd.get_dummies(df, columns=categorical_features, drop_first=True)

In [67]:
# X, y = df.drop('class', 1), df['class']
X, y = df_oh.drop('class', 1), df_oh['class']

## 1.* Модельки

In [87]:
from sklearn.ensemble import IsolationForest
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
model = IsolationForest(contamination=y_train.mean())
model.fit(X_train)
preds = model.predict(X_test)
y_test[preds == 1].mean(), y_test[preds == -1].mean()

/home/artem/anaconda3/envs/pytorch_3.6/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


(0.051759834368530024, 0.14705882352941177)

In [88]:
from sklearn.neighbors import LocalOutlierFactor
model = LocalOutlierFactor(n_neighbors=100)
preds = model.fit_predict(X_test)
y_test[preds == 1].mean(), y_test[preds == -1].mean()

/home/artem/anaconda3/envs/pytorch_3.6/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


(0.047311827956989246, 0.15384615384615385)

In [96]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
y_test[preds == 0].mean(), y_test[preds == 1].mean()

(0.057539682539682536, 0.07692307692307693)